# Download all the Python Libraries

In [ ]:
# Check if the copmuter is on google colab
import sys
if 'google.colab' in sys.modules:
    print("Running on Google Colab")
    !pip install rich
    !pip install -q -U bitsandbytes
    !pip install -q -U git+https://github.com/huggingface/transformers.git 
    !pip install -q -U git+https://github.com/huggingface/peft.git
    !pip install -q -U git+https://github.com/huggingface/accelerate.git
    !pip install datasets
    !pip install wandb
    !pip install ray[tune]
else:
    print("Not running on Google Colab")
from rich import print
import logging
from pathlib import Path
logger = logging.getLogger(__name__)
#ROOT_PATH = Path(__file__).parent.parent

# Check the GPU env
1. You can check the GPU in the Google Colab by clicking  and efficieny
2. Check if the GPU can use bfloat16 most effective as most model are pre-trained with bfloat16

In [ ]:
import torch
from rich import print
if torch.cuda.is_available():
    !nvidia-smi
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print("Cuda capability: ", torch.cuda.get_device_capability(0))
    '''
    On pre-ampere hardware bf16 works, but doesn't provide speed-ups compared to fp32 matmul operations, and some matmul operations are failing outright, so this check is more like "guaranteed to work and be performant" than "works somehow".  https://github.com/pytorch/pytorch/issues/75427
    '''
    print(f"bfloat16 support: { torch.cuda.is_bf16_supported()}") 

# Set the Seed Environment of the Notebook to ensure the reproducibility

In [ ]:
from transformers import set_seed

DEFAULT_SEED = 42

set_seed( DEFAULT_SEED )

# Constant Variable

In [ ]:
MAX_LENGTH  = 1024
BITS  = 4
LR = 1e-5
EPOCHS = 2
# Whether to use bf16 (preferred on A100's)."

# Setup Weight And Bias 💡 Configuration tips

W&B integration with Hugging Face can be configured to add extra functionalities:

* auto-logging of models as artifacts: just set environment varilable `WANDB_LOG_MODEL` to `true`
* log histograms of gradients and parameters: by default gradients are logged, you can also log parameters by setting environment variable `WANDB_WATCH` to `all`
* set custom run names with `run_name` arg present in scripts or as part of `TrainingArguments`
* organize runs by project with the `WANDB_PROJECT` environment variable

For more details refer to [W&B + HF integration documentation](https://docs.wandb.ai/integrations/huggingface).

In [ ]:
import wandb
import os

wandb.login()

%env WANDB_LOG_MODEL=true

# set the wandb project where this run will be logged
os.environ["WANDB_PROJECT"]="prompt_generator"

# save your trained model checkpoint to wandb
#@os.environ["WANDB_LOG_MODEL"]

# turn off watch to log faster
#os.environ["WANDB_WATCH"]= "all" # I am getting an error with this


# Set Up Local Training Root

# Download the Datset from the Hugging Face Datset Face Dataset

In [ ]:
DEFAULT_TRAINING_DATASET = "Rami/prompts"
from datasets import load_dataset
training_dataset = load_dataset(
    "Rami/prompts",
)
print(
    training_dataset
)


# Download the Tokenizers
1. We are suing Dolly model which was trained on the Pythia model. Instead we are recreating the dollvy tokenizer from the Pythia tokenizer

In [ ]:
from transformers import AutoTokenizer

# Special Tokens
INSTRUCTION_KEY = "### Instruction:"
INPUT_KEY = "Input:"
RESPONSE_KEY = "### Response:"
END_KEY = "### End"
RESPONSE_KEY_NL = f"{RESPONSE_KEY}\n"
DEFAULT_SEED = 42

PRETRAINED_MODEL_NAME_OR_PATH = "databricks/dolly-v2-7b"#"databricks/dolly-v2-3b"
eleutherai_python_3b = "EleutherAI/pythia-2.8b"
eleutherai_python_7b = "EleutherAI/pythia-6.9b"
dolly_v2_tokenizer = AutoTokenizer.from_pretrained(PRETRAINED_MODEL_NAME_OR_PATH)
print(dolly_v2_tokenizer)
pythia_tokenizer = AutoTokenizer.from_pretrained(eleutherai_python_7b)
print(pythia_tokenizer)

# Make sure that the pad token is the end of the tokens
pythia_tokenizer.pad_token = pythia_tokenizer.eos_token

# Add special tokens for End , Instruction , Response Key
pythia_tokenizer.add_special_tokens({
    "additional_special_tokens": [
        END_KEY,
        INSTRUCTION_KEY,
        RESPONSE_KEY,
    ]
})

print(pythia_tokenizer)

# Process the dataset
1. Convert the dataset into instruction

## Instruct Format String and Constants

In [ ]:
INTRO_BLURB = (
    "Below is an instruction that describes a task. Write a response that appropriately completes the request."
)
INSTRUCTION_KEY = "### Instruction:"
INPUT_KEY = "Input:"
RESPONSE_KEY = "### Response:"
END_KEY = "### End"
RESPONSE_KEY_NL = f"{RESPONSE_KEY}\n"
DEFAULT_SEED = 42
# This is a training prompt that does not contain an input string.  The instruction by itself has enough information
# to respond.  For example, the instruction might ask for the year a historic figure was born.
PROMPT_NO_INPUT_FORMAT = """{intro}

{instruction_key}
{instruction}

{response_key}
{response}

{end_key}""".format(
    intro=INTRO_BLURB,
    instruction_key=INSTRUCTION_KEY,
    instruction="{instruction}",
    response_key=RESPONSE_KEY,
    response="{response}",
    end_key=END_KEY,
)

# This is a training prompt that contains an input string that serves as context for the instruction.  For example,
# the input might be a passage from Wikipedia and the intruction is to extract some information from it.
PROMPT_WITH_INPUT_FORMAT = """{intro}

{instruction_key}
{instruction}

{input_key}
{input}

{response_key}
{response}

{end_key}""".format(
    intro=INTRO_BLURB,
    instruction_key=INSTRUCTION_KEY,
    instruction="{instruction}",
    input_key=INPUT_KEY,
    input="{input}",
    response_key=RESPONSE_KEY,
    response="{response}",
    end_key=END_KEY,
)

# This is the prompt that is used for generating responses using an already trained model.  It ends with the response
# key, where the job of the model is to provide the completion that follows it (i.e. the response itself).
PROMPT_FOR_GENERATION_FORMAT = """{intro}

{instruction_key}
{instruction}

{response_key}
""".format(
    intro=INTRO_BLURB,
    instruction_key=INSTRUCTION_KEY,
    instruction="{instruction}",
    response_key=RESPONSE_KEY,
)

In [ ]:
from datasets import Dataset, load_dataset

def load_training_dataset(path_or_dataset: str = DEFAULT_TRAINING_DATASET) -> Dataset:
    logger.info(f"Loading dataset from {path_or_dataset}")
    dataset = load_dataset(path_or_dataset)["train"]
    logger.info("Found %d rows", dataset.num_rows)

    def _add_text(rec):
        instruction = rec["instruction"]
        response = rec["response"]
        context = rec.get("context")

        if not instruction:
            raise ValueError(f"Expected an instruction in: {rec}")

        if not response:
            raise ValueError(f"Expected a response in: {rec}")

        # For some instructions there is an input that goes along with the instruction, providing context for the
        # instruction.  For example, the input might be a passage from Wikipedia and the instruction says to extract
        # some piece of information from it.  The response is that information to extract.  In other cases there is
        # no input.  For example, the instruction might be open QA such as asking what year some historic figure was
        # born.
        if context:
            rec["text"] = PROMPT_WITH_INPUT_FORMAT.format(instruction=instruction, response=response, input=context)
        else:
            rec["text"] = PROMPT_NO_INPUT_FORMAT.format(instruction=instruction, response=response)
        return rec

    dataset = dataset.map(_add_text)

    return dataset

## Preprocess the dataset using the tokenizers

In [ ]:
import logging
from functools import partial
from pathlib import Path
from typing import Any, Dict, List, Tuple, Union
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    DataCollatorForLanguageModeling,
    PreTrainedTokenizer,
    Trainer,
    TrainingArguments,
    set_seed,
)
def preprocess_batch(batch: Dict[str, List], tokenizer: AutoTokenizer, max_length: int) -> dict:
    return tokenizer(
        batch["text"],
        max_length=max_length,
        truncation=True,
    )
def preprocess_dataset(tokenizer: AutoTokenizer, max_length: int, seed=DEFAULT_SEED, training_dataset: str = DEFAULT_TRAINING_DATASET) -> Dataset:
    """Loads the training dataset and tokenizes it so it is ready for training.

    Args:
        tokenizer (AutoTokenizer): Tokenizer tied to the model.
        max_length (int): Maximum number of tokens to emit from tokenizer.

    Returns:
        Dataset: HuggingFace dataset
    """

    dataset = load_training_dataset(training_dataset)

    logger.info("Preprocessing dataset")
    _preprocessing_function = partial(preprocess_batch, max_length=max_length, tokenizer=tokenizer)
    dataset = dataset.map(
        _preprocessing_function,
        batched=True,
        remove_columns=["instruction", "context", "response", "text", "category"],
    )

    # Make sure we don't have any truncated records, as this would mean the end keyword is missing.
    logger.info("Processed dataset has %d rows", dataset.num_rows)
    dataset = dataset.filter(lambda rec: len(rec["input_ids"]) < max_length)
    logger.info("Processed dataset has %d rows after filtering for truncated records", dataset.num_rows)

    logger.info("Shuffling dataset")
    dataset = dataset.shuffle(seed=seed)

    logger.info("Done preprocessing")

    return dataset

In [ ]:
processed_dataset = preprocess_dataset(
    tokenizer = pythia_tokenizer,
    max_length = MAX_LENGTH,
    seed = DEFAULT_SEED,
)

# Split the Dataset

split_dataset = processed_dataset.train_test_split(test_size = 0.2, seed=DEFAULT_SEED, shuffle = True)

train_datset = split_dataset["train"]
print(train_datset)
val_dataset = split_dataset["test"]

logger.info("Train data size: %d", split_dataset["train"].num_rows)
logger.info("Test data size: %d", split_dataset["test"].num_rows)

## Data Collator For Completion Only LM

In [ ]:
import numpy as np
class DataCollatorForCompletionOnlyLM(DataCollatorForLanguageModeling):
    def torch_call(self, examples: List[Union[List[int], Any, Dict[str, Any]]]) -> Dict[str, Any]:
        batch = super().torch_call(examples)

        # The prompt ends with the response key plus a newline.  We encode this and then try to find it in the
        # sequence of tokens.  This should just be a single token.
        response_token_ids = self.tokenizer.encode(RESPONSE_KEY_NL)

        labels = batch["labels"].clone()

        for i in range(len(examples)):

            response_token_ids_start_idx = None
            for idx in np.where(batch["labels"][i] == response_token_ids[0])[0]:
                response_token_ids_start_idx = idx
                break

            if response_token_ids_start_idx is None:
                raise RuntimeError(
                    f'Could not find response key {response_token_ids} in token IDs {batch["labels"][i]}'
                )

            response_token_ids_end_idx = response_token_ids_start_idx + 1

            # Make pytorch loss function ignore all tokens up through the end of the response key
            labels[i, :response_token_ids_end_idx] = -100

        batch["labels"] = labels

        return batch

In [ ]:
if "A100" in torch.cuda.get_device_name(0):
        torch.backends.cuda.matmul.allow_tf32 = True 
else:
    torch.backends.cuda.matmul.allow_tf32 = False
# https://docs.nvidia.com/deeplearning/performance/dl-performance-matrix-multiplication/index.html
data_collator = DataCollatorForCompletionOnlyLM(
    tokenizer = pythia_tokenizer,
    mlm=False, 
    return_tensors="pt", 
    pad_to_multiple_of =   64 if torch.cuda.get_device_name(0) in ["A100-SXM4-40GB", "A100-SXM4-80GB"] else 8
)

# Download the Model
1. Torch Datat

## Setup Bits and Butes Config

## 4 Bit Configuration

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model_id = "EleutherAI/gpt-neox-20b"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    load_in_8bit = False,
    llm_int8_threshold = 6.0,
    llm_int8_has_fp16_weight=False,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype = torch.bfloat16 if torch.cuda.is_bf16_supported() else torch.float16,
)


## Download the LM Models
Then we have to apply some preprocessing to the model to prepare it for training. For that use the `prepare_model_for_kbit_training` method from PEFT.

In [ ]:
from transformers import AutoModelForCausalLM
assert torch.cuda.is_available(), "You need to have a GPU to run this notebook."
n_gpus = torch.cuda.device_count()
def model_int():
    free_in_GB = int(torch.cuda.mem_get_info()[0]/1024**3)
    max_memory = f'{int(torch.cuda.mem_get_info()[0]/1024**3)-2}GB'

    n_gpus = torch.cuda.device_count()
    max_memory = {i: max_memory for i in range(n_gpus)}
    model = AutoModelForCausalLM.from_pretrained(
        pretrained_model_name_or_path = PRETRAINED_MODEL_NAME_OR_PATH,
        trust_remote_code = True,
        use_cache = False,
        torch_dtype =  torch.bfloat16 if torch.cuda.is_bf16_supported() else torch.float16,
        device_map = "auto",
        load_in_8bit = False,
        load_in_4bit = True,
        low_cpu_mem_usage = True,
        max_memory =  max_memory,
        quantization_config = bnb_config,
    )
    return model

model = model_int()

print(model.vocab_size)

print(model)

In [ ]:
from peft import prepare_model_for_kbit_training
def print_trainable_parameters(args, model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    if args.bits == 4: trainable_params /= 2
    print(
        f"trainable params: {trainable_params} || "
        f"all params: {all_param} || "
        f"trainable: {100 * trainable_params / all_param}"
    )

model.gradient_checkpointing_enable()
try:
    model = prepare_model_for_kbit_training(model)
except:
    print("Model is already prepared for kbit training")

## Set up the LoRa Models

In [ ]:
from peft import LoraConfig, get_peft_model
import bitsandbytes as bnb
def find_all_linear_names( model):
    cls = bnb.nn.Linear4bit if BITS == 4 else (bnb.nn.Linear8bitLt if BITS == 8 else torch.nn.Linear)
    lora_module_names = set()
    for name, module in model.named_modules():
        if isinstance(module, cls):
            names = name.split('.')
            lora_module_names.add(names[0] if len(names) == 1 else names[-1])


    if 'lm_head' in lora_module_names: # needed for 16-bit
        lora_module_names.remove('lm_head')
    return list(lora_module_names)

config = LoraConfig(
    r=8, 
    lora_alpha=32, 
    target_modules = find_all_linear_names(  model),
    lora_dropout=0.05, 
    bias="none", 
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)
try:
    print_trainable_parameters(model)
except:
    print("Model is already prepared for kbit training")

## Save the PEFT Model

In [ ]:
import join
from os.path import exists, join, isdir
import transformers
class SavePeftModelCallback(transformers.TrainerCallback):
    def save_model(self, args, state, kwargs):
        print('Saving PEFT checkpoint...')
        if state.best_model_checkpoint is not None:
            checkpoint_folder = os.path.join(state.best_model_checkpoint, "adapter_model")
        else:
            checkpoint_folder = os.path.join(args.output_dir, f"{PREFIX_CHECKPOINT_DIR}-{state.global_step}")

        peft_model_path = os.path.join(checkpoint_folder, "adapter_model")
        kwargs["model"].save_pretrained(peft_model_path)

        pytorch_model_path = os.path.join(checkpoint_folder, "pytorch_model.bin")
        if os.path.exists(pytorch_model_path):
            os.remove(pytorch_model_path)

    def on_save(self, args, state, control, **kwargs):
        self.save_model(args, state, kwargs)
        return control

    def on_train_end(self, args, state, control, **kwargs):
        def touch(fname, times=None):
            with open(fname, 'a'):
                os.utime(fname, times)

        touch(join(args.output_dir, 'completed'))
        self.save_model(args, state, kwargs)

# Training Arguments 
1. WarmUp Steps 
2. Learning Rate 
   1.  0.0001 <path_or_name> https://github.com/artidoro/qlora/tree/main if the model is bigger than 13 Billions

In [ ]:
from transformers import Trainer, TrainingArguments , EarlyStoppingCallback
batch_size: int = 64 if "NVIDIA A100-SXM4-40GB" in torch.cuda.get_device_name(0) else 8 # if the model is too large for your GPU, decrease this number
micro_batch_size: int = 4
gradient_accumulation_steps = batch_size // micro_batch_size
learning_rate: float = 0.0002
epochs = 4
print(f"learning_rate: {learning_rate}")
trainer_arguments = TrainingArguments(
            output_dir = "outputs",
            per_device_train_batch_size = batch_size,
            per_device_eval_batch_size = batch_size,
            gradient_accumulation_steps=gradient_accumulation_steps,
            num_train_epochs = epochs,
            learning_rate=learning_rate,
            lr_scheduler_type = "linear",
            warmup_ratio= .3,
            bf16 =  True if torch.cuda.is_bf16_supported() else False,
            fp16 =   False if  torch.cuda.is_bf16_supported() else True,
            logging_steps = 20, # we will log every 20 steps
            eval_steps = 20, # we will evaluate every 20 steps
            evaluation_strategy="steps",
            optim = "paged_adamw_8bit",
            save_strategy="steps",
            report_to = ["wandb", "tensorboard"],
            load_best_model_at_end = True,
            metric_for_best_model="eval_loss",
            seed = DEFAULT_SEED,
            run_name = "dolly-v2-7b-prompt-generator-paged-adamw-8bit",
)

In [5]:
import math
for index in range(10):
    print(f"Training run {index}")
    #power of two
    print(math.pow(2, index))

Training run 0
1.0
Training run 1
2.0
Training run 2
4.0
Training run 3
8.0
Training run 4
16.0
Training run 5
32.0
Training run 6
64.0
Training run 7
128.0
Training run 8
256.0
Training run 9
512.0


In [ ]:
trainer = Trainer(
    model=model,
    tokenizer = pythia_tokenizer,
    args = trainer_arguments,
    train_dataset=split_dataset["train"],
    eval_dataset=split_dataset["test"],
    data_collator=data_collator,
    callbacks = [ EarlyStoppingCallback(
        early_stopping_patience = 0 , 
        )
                 ],
)

# Start training

In [ ]:
try:
    logger.info("Training")
    trainer.train()
    wandb.finish()
except Exception as e:
    print(e)

In [ ]:
train_logs = trainer.state.log_history
print(train_logs)

# Push the Model to the Hugging Face Model Hub

In [ ]:
model.push_to_hub("Rami/dolly_prompt_generator")

# Hyperparameter Tuning

## Model Init

In [ ]:
def model_int():
    return 

# References
[1] [Dolly Github](https://github.com/databrickslabs/dolly/blob/5021d941d95dddcf1f00d978d7f944709873f419/training/trainer.py#L138)
[2] https://gist.github.com/Birch-san/57878c4a27cf34f57d3e861865a7d0a2
[3] https://github.com/artidoro/qlora/blob/main/qlora.py 
[4] https://github.com/tloen/alpaca-lora/blob/main/finetune.py 